# Adjust the settings in the next cell if necessary

In [1]:
import pytesseract

# COMMENT if NOT using windows (i.e., add # before both lines)
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract"
TESSDATA_PREFIX = r"C:\Program Files\Tesseract-OCR"

# UNCOMMENT if using latest tesseract version on mac via 'brew install tesseract'
#pytesseract.pytesseract.tesseract_cmd = "/usr/local/Cellar/tesseract/4.1.1/bin/tesseract"

# Change to False if you always want your actual main stat value
assume_max_lv_gear = True

# Screenshot path. Only edit this if the screenshot directory is not within the epic7-master folder AND you know what you are doing. Otherwise just move your screenshots to the default folder.
sh_path = 'screenshots/'

# Do not change anything below here

In [2]:
from PIL import Image
from pytesseract import image_to_string

def process(img):
    resize = cv2.resize(img, (0,0), fx=5, fy=5)
    gray = cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY) # convert to gray for thresholding + blurring
    thresh = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY_INV)[1]
    blur = cv2.medianBlur(thresh, 3)
    color = cv2.cvtColor(blur, cv2.COLOR_GRAY2RGB) # convert to back to color
    img = Image.fromarray(color) # convert from array back into image format
    data = image_to_string(img, lang='eng', config='--psm 6') # PSM 6 = assume uniform block of text
    data = data.replace('Rina','Ring').replace('Edic','Epic').replace('Enic','Epic') # Fix common OCR errors
    return data

def iterative_process(img):
    # Since level and plus are read off the gear icon, there is a lot of noise
    # Thus, we need to use an iterative process to try different thresholding settings to extract a value from them
    for low in [81,100,125]:
        resize = cv2.resize(img, (0,0), fx=5, fy=5)
        gray = cv2.cvtColor(resize, cv2.COLOR_RGB2GRAY) # convert to gray for thresholding + blurring
        thresh = cv2.threshold(gray, low, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)[1]
        blur = cv2.medianBlur(thresh, 3)
        color = cv2.cvtColor(blur, cv2.COLOR_GRAY2RGB) # convert to back to color
        img = Image.fromarray(color) # convert from array back into image format
        data = image_to_string(img, lang='eng', config='--psm 7') # PSM 7 = treat as single line of text
        data = data.replace('+b','6').replace('>','0') # Fix common OCR errors
        if any(i.isdigit() for i in data): return data # If it finds a value it will terminate the loop and return the value
    return '1' # return '1' if the OCR fails to recognize the level or plus, so that there are no import errors

def stat_converter(stat):
    result = '' # default value to return just in case the stat data is bad
    if 'attack' in stat.lower():
        result = 'Atk'
        if '%' in stat: result += 'P'
    elif 'health' in stat.lower():
        result = 'HP'
        if '%' in stat: result += 'P'
    elif 'defense' in stat.lower():
        result = 'Def'
        if '%' in stat: result += 'P'
    elif 'speed' in stat.lower():
        result = 'Spd'
    elif 'chance' in stat.lower():
        result = 'CChance'
    elif 'damage' in stat.lower():
        result = 'CDmg'
    elif 'effectiveness' in stat.lower():
        result = 'Eff'
    elif 'resistance' in stat.lower():
        result = 'Res'
    return result
  

def digit_filter(val):
    # This attempts to filter out all characters that are not digits
    # If there are no digits, then it returns 0
    try: return int(''.join(filter(str.isdigit,val)))
    except ValueError: return 0

def char_filter(val):
    # This attempts to filter out all characters that are not letters
    # If there are no letters, then it returns ''
    try: return ''.join(filter(str.isalpha,val)).capitalize()
    except ValueError: return ''

def max_stat(data,item):
    stat = stat_converter(data)
    val = digit_filter(data) # Begin by setting val = actual value so that it gets returned if the rest fails
    if item['ability'] < 15: # Only change stats on items where they need to be increased
        if item['level'] in range(58,73):
            if stat == 'CChance': val = 45
            elif stat == 'CDmg': val = 55
            elif stat == 'Spd': val = 35
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 50
            elif stat == 'HP': val = 2295 # Not exactly right, finer-grained scaling
            elif stat == 'Def': val = 250 # Not exactly right, finer-grained scaling
            elif stat == 'Atk': val = 425 # Not exactly right, finer-grained scaling
        elif item['level'] in range(74,86):
            if stat == 'CChance': val = 55
            elif stat == 'CDmg': val = 65
            elif stat == 'Spd': val = 40
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 60
            elif stat == 'HP': val = 2700 # Not exactly right, finer-grained scaling
            elif stat == 'Def': val = 300 # Not exactly right, finer-grained scaling
            elif stat == 'Atk': val = 500 # Not exactly right, finer-grained scaling
        elif item['level'] in range(87,89):
            if stat == 'CChance': val = 60
            elif stat == 'CDmg': val = 70
            elif stat == 'Spd': val = 45
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 65
            elif stat == 'HP': val = 2765
            elif stat == 'Def': val = 310
            elif stat == 'Atk': val = 515
    return val

In [3]:
from glob import glob
from string import ascii_lowercase, digits
import random
import json
import cv2

# Format the json for the optimizer
export = {'processVersion':'1','heroes':[],'items':[]}
temp_list = []

print('Beginning export process...')

# Gather the filenames for all the screenshots. Looks for png, jpg, and jpeg.
filenames = glob(sh_path+'*.png')
filenames.extend(glob(sh_path+'*.jpg'))
filenames.extend(glob(sh_path+'*.jpeg'))

did_break = False
errors = 0
for n,name in enumerate(filenames):
    
    # The height of the item box changes depending on the length of the item and set descriptions,
    # we have to crop the top and bottom info separately in order to ensure the OCR boxes within these areas
    # remain in fixed locations. we then process the top and bottom info independently.
    
    print('currently processing '+name.replace('screenshots\\',''))
    img = cv2.imread(name)
    
    # Make sure the screenshot has the proper dimensions
    if img.shape[0] != 1080 or img.shape[1] != 2200:
        print('ERROR: This screenshot is not 2200x1080!')
        print('Please retake the screenshot in Nox or LDPlayer with the internal resolution set to 2200x1080')
        did_break = True
        break
    
    # Setup dictionary for the current item
    item = {'locked':False,'efficiency':0}
    
    # Settings for how the boxes are cropped. If the UI changes, these may need to be updated
    width = [725,1190] # First = pixel that the gearbox starts at, Second = pixel that the gearbox ends at
    top_depth = 160 # This is how many pixels deep the top cropped image will be
    bottom_offset = 25 # This is how many pixels we start the bottom box from the divider in the gear screenshot
    bottom_depth = 335 # This is how many pixels deep the bottom cropped image will be
    
    # Coordinates for the various substat types WITHIN the cropped boxes detailed above. If the UI changes, these may need to be updated
    # Format = {'stat': [[X1,X2],[Y1,Y2]]} as this marks the coordinates for a mini box for that individual stat
    # top_coords are all *relative* to the dimensions of the top box
    top_coords = {'type': [[20,60],[172,432]],
                  'level': [[19,47],[35,69]],
                  'plus': [[2,26],[132,161]]}
    # bot_coords are all *relative* to the dimensions of the bottom box
    bot_coords = {'main': [[8,70],[65,435]],
                  'subs': [[98,255],[25,435]],
                  'set': [[280,340],[76,435]]}
    
    # Top box
    temp_top = cv2.imread('e7/top.jpg',0)
    a, b, c, max_loc = cv2.minMaxLoc(cv2.matchTemplate(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), temp_top, cv2.TM_CCOEFF_NORMED))
    top_box = img[max_loc[1]:max_loc[1]+top_depth,width[0]:width[1]]

    # Bottom box
    temp_bot = cv2.imread('e7/bottom.jpg',0)
    a, b, c, max_loc = cv2.minMaxLoc(cv2.matchTemplate(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), temp_bot, cv2.TM_CCOEFF_NORMED))
    bottom_box = img[max_loc[1]+bottom_offset:max_loc[1]+bottom_offset+bottom_depth,width[0]:width[1]]
    
    # Process stats from the Top box
    # Type
    data = process(top_box[top_coords['type'][0][0]:top_coords['type'][0][1],top_coords['type'][1][0]:top_coords['type'][1][1]])
    split_data = data.split(' ')
    item['rarity'] = char_filter(split_data[0])
    item['slot'] = char_filter(split_data[1].split('\n')[0])
    if len(item['rarity']) == 0: print('Error: no rarity detected')
    if len(item['slot']) == 0: print('Error: no slot detected')
    
    # Level
    data = iterative_process(top_box[top_coords['level'][0][0]:top_coords['level'][0][1],top_coords['level'][1][0]:top_coords['level'][1][1]])
    data = data.replace('S','5').replace('B','8').replace('a','8') # Fix common OCR errors
    item['level'] = digit_filter(data)
    
    # Enhance lvl (plus)
    data = iterative_process(top_box[top_coords['plus'][0][0]:top_coords['plus'][0][1],top_coords['plus'][1][0]:top_coords['plus'][1][1]])
    data = data.replace('S','5').replace('B','8').replace('a','8') # Fix common OCR errors
    item['ability'] = digit_filter(data)

    print("ilvl", item['level'],"/ enhanced to +"+str(item['ability'])+" / slot:",item['slot']," / rarity:",item['rarity'])
    
    # Process stats from Bottom box    
    # Main
    data = process(bottom_box[bot_coords['main'][0][0]:bot_coords['main'][0][1],bot_coords['main'][1][0]:bot_coords['main'][1][1]])
    stat = stat_converter(data)
    if assume_max_lv_gear is True: item['mainStat'] = [stat,max_stat(data,item)]
    else: item['mainStat'] = [stat,digit_filter(data)]
    
    # Subs
    data = process(bottom_box[bot_coords['subs'][0][0]:bot_coords['subs'][0][1],bot_coords['subs'][1][0]:bot_coords['subs'][1][1]])
    for n,entry in enumerate(data.split('\n')):
        if n < 4:
            stat = stat_converter(entry)
            entry = entry.replace('T','7') # Fix common OCR error
            val = digit_filter(entry)
            if len(stat) > 0 and val != 0:
                item['subStat'+str(n+1)] = [stat,val]
    
    # Set
    data = process(bottom_box[bot_coords['set'][0][0]:bot_coords['set'][0][1],bot_coords['set'][1][0]:bot_coords['set'][1][1]])
    data_split = data.split(' Set')
    item['set'] = char_filter(data_split[0])
    if len(item['set']) == 0: print('Error: no set detected')
    
    # Check to make sure the item does not already exist in the item dictionary
    # Also verifies that it has a valid slot and rarity entry
    # If these conditions are met, this assigns the item a unique ID and adds it to the item dictionary for export
    if item not in temp_list and len(item['slot']) > 0 and len(item['rarity']) > 0:
        temp_list.append(item)
        item['id'] = 'jt'+''.join(random.choice(digits+ascii_lowercase) for _ in range(6))
        export['items'].append(item)
        print(len(export['items']),"exported out of",len(filenames)-errors,'valid items \n')
    else:
        errors += 1
        print('Item not exported because of fatal errors in processing it \n')

# Export dictionary to json for importing into optimizer if everything completed properly
if not did_break:
    with open('exported_gear.json', 'w') as f: json.dump(export, f)
    print('JSON file finished!')

Beginning export process...
currently processing screenshots/test_screenshot.png
ilvl 88 / enhanced to +15 / slot: Weapon  / rarity: Epic
1 exported out of 1 valid items 

JSON file finished!
